<h1 style="text-align:center;">Setting  remote syslog server</h1>

<div class="alert alert-block alert-info" style="border-radius: 20px;">
Welcome to <b>python-iLOrest-library</b> <br> Here is an example to <b>Setting remote syslog server</b><br>
    This Jupyter Notebook will contains a example code to demonstrate how to set remote syslog server. This code is written in Python and uses the redfish library to perform the required operations</b><br> As you are here let's get it done easily.<br>
    To perform this you need to have <a href = "https://libraries.io/pypi/python-ilorest-library#id3" style = "none">python-ilorest-library</a> installed in your machine.<br>
    Please keep your username and password handy!
    <br>
    <b>Also do execute each successive cell once to avoid any errors!</b>
    
</div>

<div class="alert alert-block alert-success" style="border-radius: 20px;">
    <h3> 1. Below we are importing all the necessary modules from the ilorest library.</h3><br>
    <li><a href = "https://www.tutorialspoint.com/what-is-python-s-sys-module#:~:text=%20What%20is%20Python%27s%20Sys%20Module%20%201,%5B0%5D%20prints%20the...%205%20Output.%20%20More%20">sys </a>
    </li>
    <li>json </li>
    <li> redfish </li>
    <li> ilorest_util</li>
    <li> argparse </li> 
</div>

In [ ]:
import sys
import json
import argparse
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError
global DISABLE_RESOURCE_DIR
from ilorest_util import get_resource_directory
from ilorest_util import get_gen

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <h3> 2. This is our set_syslog function overview</h3><br>
    This function sets the remote syslog server and enables remote syslog on an iLO system. <br><br>
1. It takes two parameters:
<ul>
    <li><b>_redfishobj:</b> an object representing the connection to the iLO device through the Redfish API.</li>
    <li><b>syslog_server:</b> a string representing the IP address or hostname of the remote syslog server.</li></ul><br>
2. The function first retrieves the model information of the system, which is used to determine the Oem (Original equipment manufacturer) name, as the endpoint structure and payload might differ based on the manufacturer. It determines the model of the server by sending a GET request to the <b>/redfish/v1/Systems/1/</b> URI and extracting the <b>Model</b> attribute from the response.<br><br>
3. It checks if the model contains "Gen9" or "Gen8". If either of them is present, it sets the "hp" variable to "Hp". Otherwise, it sets it to "Hpe".<br><br>
4. It then constructs the URI for the syslog configuration resource and sends a PATCH request with the request body to set the remote syslog server and enable remote syslog. <br><br>
    5. Finally, it calls the <b>ilo_response</b>, passing the <b>_redfishobj</b> and <b>resp</b> (response) as arguments. This function handles response from the server.<br><br>
</div>

In [ ]:
def set_syslog(_redfishobj, syslog_server):

    model_uri = "/redfish/v1/Systems/1/"
    model = _redfishobj.get(model_uri).obj['Model']
    print (model)
    if "Gen9" in model or 'Gen8' in model:
        hp = "Hp"
    else:
        hp = "Hpe"

    syslog_uri = "/redfish/v1/Managers/1/NetworkService/"

    body = {"Oem": {hp: {"RemoteSyslogServer": syslog_server, "RemoteSyslogEnabled": True}}}
    resp = _redfishobj.patch(syslog_uri, body)
    ilo_response(_redfishobj, resp)

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <h3> 3. This is ilo_response function overview</h3><br>
    The ilo_response function is responsible for handling the response from the iLO server.<br><br>
1. It takes two parameters: <b>_redfishobj</b> (Redfish client object) and <b>resp</b> (response object).<br><br>
2. It checks the status code of the response. If it is equal to 400, it indicates a client error.<br><br>
3. It tries to extract and print the extended error information from the response using the key error <b>['@Message.ExtendedInfo']</b>.
If there is an exception while accessing the extended error information, it writes an error message to the standard error stream.<br><br>
4. If the status code is not 400 but is not equal to 200 either, it indicates an HTTP response error.
It writes an error message to the standard error stream, indicating the HTTP response status.<br><br>
5. If the status code is 200, it indicates a successful response.
It prints the message "Success" to indicate the operation was successful.<br>
    </div>

In [ ]:
def ilo_response(_redfishobj, resp):

    if resp.status == 400:
        try:
            print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], indent=4, \
                                                                         sort_keys=True))
        except Exception as excp:
            sys.stderr.write("A response error occurred, unable to access iLO Extended "\
                             "Message Info...")
    elif resp.status != 200:
        sys.stderr.write("An http response of \'%s\' was returned.\n" % resp.status)
    else:
        print("Success")

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <h3> 4. This is Main function overview.<br></h3>
1. An argument parser is initialized to parse the command-line arguments passed to the script. The parser expects arguments such as iLO url, username, password, and syslog server IP.<br><br>
2.The rest opertions are handled by Redfish object which is created by Redfish Client<br><br>
3. <b>Redfish Client</b> class takes 3 parameters<i> iLO url, Username, Password</i> and returns a redfish object. Once you enter everything a redfish object is created with the help of Redfish Client class.<br><br>
4. Now we try to Login with the help of <i> REDFISHOBJ</i>, if it is successful a session is created and you are logged in, But if Server is not avaialbe we get an error saying "server unreachable"<br><br>
5. The <b>get_gen</b> function is called to retrieve the iLO generation number of the server.
    Depending on the iLO generation number, the <b>set_syslog</b> function is called with the Redfish client object and the syslog server IP address.<br><br>
6. <b>REDFISHOBJ.logout()</b> will take care of logging you out of the iLO.

 </div>

In [ ]:
if __name__ == "__main__":
    # Initialize parser
    parser = argparse.ArgumentParser(description = "Script to upload and flash NVMe FW")

    parser.add_argument(
        '-i',
        '--ilo',
        dest='ilo_ip',
        action="store",
        help="iLO IP of the server",
        default=None)
    parser.add_argument(
        '-u',
        '--user',
        dest='ilo_user',
        action="store",
        help="iLO username to login",
        default=None)
    parser.add_argument(
        '-p',
        '--password',
        dest='ilo_pass',
        action="store",
        help="iLO password to log in.",
        default=None)
    parser.add_argument(
        '-s',
        '--syslog_server',
        dest='syslog_server',
        action="store",
        required=True,
        help="iLO password to log in.",
        default=None)

    options = parser.parse_args()

    system_url = "https://" + options.ilo_ip
    print (system_url)

    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = True

    try:
        # Create a Redfish client object
        redfish_obj = RedfishClient(base_url=system_url, username=options.ilo_user, password=options.ilo_pass)
        # Login with the Redfish client
        redfish_obj.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    (ilogen,_) = get_gen(redfish_obj)
    print ("Generation is ", ilogen)
    if int(ilogen) == 5:
        set_syslog(redfish_obj, options.syslog_server)
    else:
        set_syslog(redfish_obj, options.syslog_server)
    redfish_obj.logout()


<div class="alert alert-block alert-warning" style="border-radius: 20px;">
<b> When running on the server locally : </b>  use the following values <br>
    SYSTEM_URL = None <br>
    LOGIN_ACCOUNT = None <br>
    LOGIN_PASSWORD = None <br>
    <b> When running remotely </b> connect using the secured (https://) address,account name, and password to send https requests <br>
    SYSTEM_URL acceptable examples: <br>
    "https://10.0.0.100" <br>
    "https://ilo.hostname"
</div>

<div class = "alert alert-block alert-success" style="border-radius: 20px;">In case you need help 🔗 links to relevant resources :
    <br>
    1.Python-ilorest-library:<a href = "https://github.com/HewlettPackard/python-ilorest-library">LINK</a><br>
    2.Hpe ilorest-api explorer:<a href = "https://ilorestfulapiexplorer.ext.hpe.com/">LINK</a><br>
    3.Rest api: <a href = "https://restfulapi.net/">LINK</a><br>
    
</div>